### import

In [1]:
import torch
from torch import nn
from torch.functional import F
from torch.distributions import Multinomial, Categorical
from torch import optim
import numpy as np
import importlib as imp
import actor
import envs
import utils
imp.reload(actor)
imp.reload(envs)
imp.reload(utils)
from actor import Actor, Critic, Actor_GRU
from envs import HospitalEnv
from utils import mlp_sample, rnn_sample, ppo_update

In [2]:
%load_ext autoreload
%autoreload 2
# %autoreload 1 # 只reload import了的module
# %autoreload 0 # 关闭autoreload

### 2池

In [497]:
J = 2
lam = [50, 50]
m = 8
env = HospitalEnv(J=J, lam=[6.25, 6.25], N=[28, 32], C=[24, 24], B=[[0,90],[90,0]], mu=[0.1, 0.1], m=m)

new_actor = Actor(J=J)
old_actor = Actor(J=J)
old_actor.load_state_dict(new_actor.state_dict())
crictic = Critic(J=J)


In [466]:
trajectory = mlp_sample(env, old_actor, max_steps=50)

In [500]:
optimizer_actor = optim.Adam(new_actor.parameters(), lr=1e-3)
optimizer_crictic = optim.Adam(crictic.parameters(), lr=1e-3)
gamma=0.99
clip_ratio=0.2
train_epochs=5
batch_size=10
states = torch.stack([t['state'] for t in trajectory]).float()
actions = torch.stack([t['action'] for t in trajectory])  # 动作是f[i][j]数组
rewards = torch.stack([t['reward'] for t in trajectory])
next_states = torch.stack([t['next_state'] for t in trajectory]).float()
f = torch.stack([t['f'] for t in trajectory])
old_probs = torch.stack([t['probs'] for t in trajectory])
memory = states, actions, old_probs, rewards, next_states, f

In [523]:
ppo_update(new_actor, crictic, memory, optimizer_actor, optimizer_crictic,gamma, clip_ratio, batch_size)

### 5池

#### 1.env actor初始化

In [2]:
from utils import arrival_rate_5, discharge_probability_5
J = 5
m = 24
N = [60, 64, 67, 62, 62]
mu = [0.25] * J
# mask = [[1,1,1,0,1],[0,1,1,1,1],[1,1,1,0,1],[1,1,1,1,0],[1,1,1,0,1]] B可取代mask
B = [[1,35,35,0,30],[0,1,30,35,35],[35,30,1,0,35],[35,30,35,1,0],[30,35,35,0,1]] # 0是不可overflow的科室, 1是自己
C = [6] * J
env = HospitalEnv(J=J, lam=arrival_rate_5, N=N, C=C, B=B, mu=mu, m=m, discharge_probability=discharge_probability_5)
crictic = Critic(J=J)

In [4]:
state = env.reset(True)

In [5]:
state

tensor([ 0, 44, 52, 58, 49, 55,  0,  0,  0,  0,  0])

In [8]:
logits = old_actor(state.float())
logits, logits.shape

(tensor([[[ 8.8019e+00,  3.9804e+01,  5.3562e+01, -1.0000e+09,  2.2944e+01],
          [-1.0000e+09,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.3006e+01],
          [ 0.0000e+00,  8.8376e+01,  1.9907e+02, -1.0000e+09,  0.0000e+00],
          [ 0.0000e+00,  3.6994e+00,  1.4780e+01,  0.0000e+00, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  1.0053e+01, -1.0000e+09,  6.0628e+01]]],
        grad_fn=<MaskedFillBackward0>),
 torch.Size([1, 5, 5]))

In [9]:
prob = F.softmax(logits, dim=-1)
prob, prob.shape

(tensor([[[3.6380e-20, 1.0587e-06, 1.0000e+00, 0.0000e+00, 5.0422e-14],
          [0.0000e+00, 2.1018e-19, 2.1018e-19, 2.1018e-19, 1.0000e+00],
          [0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00],
          [3.8123e-07, 1.5410e-05, 9.9998e-01, 3.8123e-07, 0.0000e+00],
          [4.6719e-27, 4.6719e-27, 1.0850e-22, 0.0000e+00, 1.0000e+00]]],
        grad_fn=<SoftmaxBackward0>),
 torch.Size([1, 5, 5]))

In [ ]:
env()

In [3]:
new_actor = Actor(J=J, B=B)
old_actor = Actor(J=J, B=B)

In [40]:
new_actor = Actor_GRU(J=J, B=B)
old_actor = Actor_GRU(J=J, B=B)

#### 3.训练

In [41]:
optimizer_actor = optim.Adam(new_actor.parameters(), lr=1e-3)
optimizer_crictic = optim.Adam(crictic.parameters(), lr=1e-3)
gamma=0.99
clip_ratio=0.2
train_epochs=5
batch_size=24

In [42]:
old_actor.load_state_dict(new_actor.state_dict())
trajectory = rnn_sample(env, old_actor, max_steps=24, is_random=True)
states = torch.stack([t['state'] for t in trajectory]).float()
actions = torch.stack([t['action'] for t in trajectory])  # 动作是f[i][j]数组
rewards = torch.stack([t['reward'] for t in trajectory])
next_states = torch.stack([t['next_state'] for t in trajectory]).float()
f = torch.stack([t['f'] for t in trajectory])
old_probs = torch.stack([t['probs'] for t in trajectory])
print(f"平均cost{rewards.float().mean()}")
memory = states, actions, old_probs, rewards, next_states, f

平均cost-8.416666984558105


In [43]:
ppo_update(new_actor, crictic, memory, optimizer_actor, optimizer_crictic, gamma, clip_ratio, batch_size)

PPO Update:   0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'softmax'

In [14]:
for i in range(train_epochs):
    print(f"epoch : {i+1}")
    old_actor.load_state_dict(new_actor.state_dict())
    trajectory = sample_episode(env, old_actor, max_steps=2400, is_random=True)
    states = torch.stack([t['state'] for t in trajectory]).float()
    actions = torch.stack([t['action'] for t in trajectory])  # 动作是f[i][j]数组
    rewards = torch.stack([t['reward'] for t in trajectory])
    next_states = torch.stack([t['next_state'] for t in trajectory]).float()
    f = torch.stack([t['f'] for t in trajectory])
    old_probs = torch.stack([t['probs'] for t in trajectory])
    print(f"平均cost{rewards.float().mean()}")
    memory = states, actions, old_probs, rewards, next_states, f
    ppo_update(new_actor, crictic, memory, optimizer_actor, optimizer_crictic, gamma, clip_ratio, batch_size)

epoch : 1
平均cost-22.237083435058594


PPO Update:   0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: Size does not match at dimension 0 expected index [24, 5, 1] to be smaller than self [1, 5, 5] apart from dimension 2

In [13]:
trajectory = sample_episode(env, old_actor, max_steps=100000)

In [16]:
trajectory

[{'state': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'action': tensor([4, 2, 0, 1, 1]),
  'probs': tensor([[0.2701, 0.2833, 0.2233, 0.0000, 0.2233],
          [0.0000, 0.2445, 0.2664, 0.2445, 0.2445],
          [0.2293, 0.2293, 0.2293, 0.0000, 0.3122],
          [0.2380, 0.2393, 0.2846, 0.2380, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.0000, 0.2500]]),
  'reward': tensor(0),
  'f': tensor([0, 0, 0, 0, 0]),
  'next_state': tensor([1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0])},
 {'state': tensor([1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
  'action': tensor([4, 3, 0, 2, 4]),
  'probs': tensor([[0.2366, 0.2903, 0.2366, 0.0000, 0.2366],
          [0.0000, 0.2004, 0.2224, 0.2852, 0.2920],
          [0.1621, 0.1621, 0.3007, 0.0000, 0.3751],
          [0.2951, 0.1934, 0.3182, 0.1934, 0.0000],
          [0.2418, 0.2745, 0.2418, 0.0000, 0.2418]]),
  'reward': tensor(0),
  'f': tensor([0, 0, 0, 0, 0]),
  'next_state': tensor([2, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0])},
 {'state': tensor([2, 3, 0, 0, 0, 2, 0, 0, 0, 

### 10池

In [ ]:
J = 10
N = [39, 43, 46, 41, 41, 81, 85, 88, 83, 83]

env = HospitalEnv(J=J, lam=[6.25, 6.25], N=N, C=[24, 24], B=[[0,90],[90,0]], mu=[0.1, 0.1], m=m)